<a href="https://colab.research.google.com/github/SanzharIbragimov/cooffeetime.github.io/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Tweets.csv to Tweets.csv


In [ ]:
from google.colab import drive

drive.mount('/content/drive')
model.save('/content/drive/MyDrive/sentiment_model.h5')

Mounted at /content/drive


In [161]:
import tensorflow as tf
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.utils import pad_sequences


label_dict = {'negative': 0, 'neutral': 1, 'positive': 2}

# Load the dataset8
df = pd.read_csv('Tweets.csv')

# Preprocess the data
df = df[['text', 'airline_sentiment']]
df['airline_sentiment'] = df['airline_sentiment'].map(lambda x: label_dict[x])

train_df = df.iloc[:14000]
val_df = df.iloc[14000:]

tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_df['text'])

train_sequences = tokenizer.texts_to_sequences(train_df['text'])
val_sequences = tokenizer.texts_to_sequences(val_df['text'])

max_len = 100
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post', truncating='post')
val_padded = pad_sequences(val_sequences, maxlen=max_len, padding='post', truncating='post')

train_labels = to_categorical(train_df['airline_sentiment'])
val_labels = to_categorical(val_df['airline_sentiment'])

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=16, input_length=max_len),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_padded, train_labels, validation_data=(val_padded, val_labels), epochs=10)

# Save the model
model.save('sentiment_model.h5')


Epoch 1/10
438/438 [==============================] - 2s 4ms/step - loss: 0.9379 - accuracy: 0.6136 - val_loss: 0.6989 - val_accuracy: 0.7797
Epoch 2/10
438/438 [==============================] - 2s 4ms/step - loss: 0.7724 - accuracy: 0.6573 - val_loss: 0.5239 - val_accuracy: 0.8234
Epoch 3/10
438/438 [==============================] - 2s 4ms/step - loss: 0.6241 - accuracy: 0.7524 - val_loss: 0.4517 - val_accuracy: 0.8422
Epoch 4/10
438/438 [==============================] - 2s 4ms/step - loss: 0.5444 - accuracy: 0.7990 - val_loss: 0.4183 - val_accuracy: 0.8516
Epoch 5/10
438/438 [==============================] - 2s 4ms/step - loss: 0.4842 - accuracy: 0.8231 - val_loss: 0.3900 - val_accuracy: 0.8484
Epoch 6/10
438/438 [==============================] - 2s 5ms/step - loss: 0.4373 - accuracy: 0.8417 - val_loss: 0.3871 - val_accuracy: 0.8422
Epoch 7/10
438/438 [==============================] - 2s 4ms/step - loss: 0.3980 - accuracy: 0.8569 - val_loss: 0.3737 - val_accuracy: 0.8531
Epoch 

In [166]:
import tensorflow as tf
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.utils import pad_sequences
from keras.layers import Dense

label_dict = {'negative': 0, 'neutral': 1, 'positive': 2}

# Load the validation dataset
val_df = val_df
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(val_df['text'])

model = tf.keras.models.load_model('sentiment_model.h5')
model.add(Dense(3, activation='softmax'))

# Preprocess the data
val_sequences = tokenizer.texts_to_sequences(val_df['text'])
val_padded_sequences = pad_sequences(val_sequences, maxlen=100, padding='post', truncating='post')
val_labels = to_categorical(val_df['airline_sentiment'].map(lambda x: label_dict.get(x, 0)), num_classes=3)

# Evaluate the model on the validation dataset
loss, accuracy = model.evaluate(val_padded_sequences, val_labels, verbose=1)

print('Test loss:', loss)
print('Test accuracy:', accuracy)


20/20 [==============================] - 0s 2ms/step - loss: 0.6474 - accuracy: 0.9766
Test loss: 0.6474207043647766
Test accuracy: 0.9765625


In [150]:
import tensorflow as tf
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.utils import pad_sequences
from keras.layers import Dense

label_dict = {'negative': 0, 'neutral': 1, 'positive': 2}

# Load the validation dataset
val_df = val_df
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(val_df['text'])

model = tf.keras.models.load_model('sentiment_model.h5')
model.add(Dense(3, activation='softmax'))

# Preprocess the data
val_sequences = tokenizer.texts_to_sequences(val_df['text'])
val_padded_sequences = pad_sequences(val_sequences, maxlen=100, padding='post', truncating='post')
val_labels = to_categorical(val_df['airline_sentiment'].map(lambda x: label_dict.get(x, 0)), num_classes=3)

# Evaluate the model on the validation dataset
loss, accuracy = model.evaluate(val_padded_sequences, val_labels, verbose=1)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

# Predict sentiment for a new text
new_text = "This is a great airline!"
new_sequence = tokenizer.texts_to_sequences([new_text])
new_padded_sequence = pad_sequences(new_sequence, maxlen=100, padding='post', truncating='post')
prediction = model.predict(new_padded_sequence)[0]
sentiment = list(label_dict.keys())[prediction.argmax()]

print('New text: {}'.format(new_text))
print('New text: {}'.format(prediction))
print('Sentiment prediction: {}'.format(sentiment))


20/20 [==============================] - 0s 2ms/step - loss: 1.2152 - accuracy: 0.0938
Test loss: 1.2151813507080078
Test accuracy: 0.09375
1/1 [==============================] - 0s 66ms/step
New text: This is a great airline!
New text: [0.2814993 0.2040241 0.5144766]
Sentiment prediction: positive


In [118]:
from keras.models import load_model

model = load_model('sentiment_model.h5')
import pandas as pd

test_data = val_df #pd.read_csv('unlabeled_data.csv')

from keras.preprocessing.text import Tokenizer
# from keras_preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(test_data['text'])

sequences = tokenizer.texts_to_sequences(test_data['text'])
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')
predictions = model.predict(padded_sequences)


20/20 [==============================] - 0s 2ms/step


In [119]:
sentiment = []

for pred in predictions:
    if pred.argmax() == 0:
        sentiment.append('negative')
    elif pred.argmax() == 1:
        sentiment.append('neutral')
    else:
        sentiment.append('positive')

In [120]:
print(predictions)

print(sentiment)
print(len(sentiment))
print(len(test_data))
print(len(df.iloc[14000:]))

[[1.2002832e-01 3.8985059e-01 4.9012110e-01]
 [9.9156880e-01 7.7998610e-03 6.3131592e-04]
 [7.8519091e-02 4.7759846e-01 4.4388238e-01]
 ...
 [7.2801512e-01 2.2503191e-01 4.6953004e-02]
 [9.9938893e-01 1.4804884e-04 4.6297684e-04]
 [9.9263352e-01 3.9316965e-03 3.4347738e-03]]
['positive', 'negative', 'neutral', 'negative', 'negative', 'negative', 'negative', 'neutral', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'neutral', 'negative', 'positive', 'neutral', 'negative', 'neutral', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'neutral', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'neutral', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'neutral', 'negative',